In [1]:
import joblib
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from SALib.sample import saltelli
from SALib.analyze import sobol
import GPy
import contextlib
!pwd

/Users/pmzff/Documents/GitHub/ModularCircFF/ExploreModularCirc


### Emulator functions

In [2]:
def R2(df_xtest, df_ytest, model):
    pred_mean, pred_var = model.predict(df_xtest) # is it prediction variance or SD?
    MSE = np.mean((pred_mean-df_ytest)**2)
    R2=1.-MSE/np.var(df_ytest)
    return(R2)

In [3]:
def emulate_RBF(input, output, percentage, restarts, suppress=False):
    # Calculate the number of rows to sample (20% of the total rows)
    sample_size = int(len(input) * percentage)

    # Randomly select indices (using random_state for reproducibility)
    sample_indices = input.sample(n=sample_size, random_state=42).index
    remaining_indices = input.index.difference(sample_indices)

    # Test Data
    xtest = input.loc[sample_indices].values
    ytest = output.loc[sample_indices].values

    # Training Data
    xtrain = input.loc[remaining_indices].values
    ytrain = output.loc[remaining_indices].values

    # Define the kernel
    kernel = GPy.kern.RBF(input_dim=xtrain.shape[1], ARD=True)

    # Create the GP model
    model = GPy.models.GPRegression(xtrain, ytrain, kernel)

    # Optimize the model with optional print suppression
    if suppress:
        with open(os.devnull, 'w') as f, contextlib.redirect_stdout(f):
            model.optimize_restarts(restarts, messages=False, num_processes=0)
    else:
        model.optimize_restarts(restarts, messages=False)

    # Print model parameters
    if not suppress:
        print(model.kern.lengthscale)
        print(model)
        for _ in range(3):
            print()
        print(f'R2 = {R2(xtest, ytest, model)}')

    # Compute R² score for the predictions versus actual test data
    r2 = R2(xtest, ytest, model)
    return model, r2

### Load input and output data

In [4]:
n_runs=5000
main_path = os.getcwd()
main_path

'/Users/pmzff/Documents/GitHub/ModularCircFF/ExploreModularCirc'

In [5]:
# Read Input Data
df_x = pd.read_csv(f'{main_path}/Input/input_{n_runs}_converged.csv')

# Select relevant inputs only
relevant_columns = []
for col in df_x.columns:
    relevant_columns.append(col)
    if col == 'T': break

#columns_with_multiple_values = df_x.nunique() > 1
#filtered_input = df_x.loc[:, columns_with_multiple_values]

# Select only first 5 inputs 
filtered_input = df_x[relevant_columns]

filtered_input


,# sas.r,sas.c,sas.l,sat.r,sat.c,sat.l,svn.r,svn.c,pas.r,pas.c,...,la.k_pas,rv.E_pas,rv.E_act,rv.v_ref,rv.k_pas,ra.E_pas,ra.E_act,ra.v_ref,ra.k_pas,T
0,0.004052,0.114509,0.000053,0.925069,1.023913,0.001803,0.069190,18.228856,0.001374,0.245804,...,0.013339,0.958286,0.818308,45.774977,0.010605,0.736641,0.308665,15.185054,0.020651,0.718265
1,0.001690,0.072214,0.000067,1.558170,1.846973,0.001607,0.088602,30.052234,0.002245,0.139568,...,0.026536,0.569621,2.045156,22.548241,0.024189,0.287757,0.201274,23.815151,0.010173,0.415870
2,0.002818,0.089454,0.000044,0.593984,2.035777,0.001217,0.052570,22.333355,0.002551,0.098213,...,0.020185,0.823862,3.372404,54.036097,0.018730,0.521606,0.170235,26.143477,0.016704,0.892658
3,0.003680,0.047260,0.000092,1.235169,1.211961,0.002196,0.108168,10.414924,0.001922,0.191447,...,0.019632,0.369782,1.814099,37.098420,0.027335,0.440337,0.340188,14.700294,0.027480,0.660382
4,0.003279,0.090641,0.000076,1.092825,1.574651,0.002106,0.042848,24.249495,0.002967,0.173215,...,0.028348,0.896731,1.069150,57.500446,0.013318,0.388163,0.147805,21.655356,0.013636,0.486058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4980,0.003680,0.042290,0.000053,1.321360,0.811489,0.000927,0.045077,19.707024,0.002716,0.129187,...,0.025942,0.913201,2.372831,36.693376,0.013438,0.737160,0.285333,17.514775,0.022053,0.823187
4981,0.003237,0.095746,0.000038,1.198833,1.173799,0.001674,0.110938,27.768776,0.001673,0.237409,...,0.017313,0.353393,3.115920,58.706576,0.027157,0.684843,0.233575,25.715870,0.018396,0.717560
4982,0.002598,0.058468,0.000082,0.766095,1.997614,0.001710,0.054179,16.510909,0.002802,0.142241,...,0.022854,0.796984,1.889086,30.470227,0.018263,0.353269,0.278467,14.659077,0.027584,0.415489
4983,0.002226,0.100719,0.000059,1.394404,2.185274,0.002156,0.086115,14.054665,0.002496,0.095354,...,0.029178,0.554602,0.743351,41.656416,0.024054,0.586950,0.372493,15.455727,0.024295,0.893030


In [6]:
# Import output data 

dataframes = {}

# Read PCA data
for i in range(3):
    df_name = f'PC{i+1}'  # Create the dataframe name
    dataframes[f'press_only_{df_name}'] = pd.read_csv(f'{main_path}/Outputs//Output_5000/PCA/press_only_PC{i+1}.csv')  # Read and store the dataframe
    dataframes[f'press_CO_{df_name}'] =  pd.read_csv(f'{main_path}/Outputs/Output_5000//PCA/press_CO_PC{i+1}.csv') 
    dataframes[f'press_CO_dt_{df_name}'] =  pd.read_csv(f'{main_path}/Outputs/Output_5000/PCA/press_CO_dt_PC{i+1}.csv') 

press_only_PC1 = dataframes['press_only_PC1']
press_only_PC2 = dataframes['press_only_PC2']
press_only_PC3 = dataframes['press_only_PC3']

press_CO_PC1 = dataframes['press_CO_PC1']
press_CO_PC2 = dataframes['press_CO_PC2']
press_CO_PC3 = dataframes['press_CO_PC3']

press_CO_dt_PC1 = dataframes['press_CO_dt_PC1']
press_CO_dt_PC2 = dataframes['press_CO_dt_PC2']
press_CO_dt_PC3 = dataframes['press_CO_dt_PC3']

df_pressure = pd.read_csv(f'{main_path}/Outputs/Output_5000/pressure_traces/all_pressure_traces.csv')
dataframes['CO'] = df_pressure.iloc[:,100:101]

dataframes['mean_press']= df_pressure.iloc[:,:100].mean(axis=1).to_frame(name='mean_press')
dataframes['max_press']= df_pressure.iloc[:,:100].max(axis=1).to_frame(name='max_press')
dataframes['min_press']= df_pressure.iloc[:,:100].min(axis=1).to_frame(name='min_press')


### Run Emulator 

In [9]:
X = filtered_input

test_perc = 0.2 
restarts = 10

emulate_RBF(input=filtered_input, output=dataframes['max_press'], percentage=test_perc, restarts=restarts, suppress=False)

Optimization restart 1/10, f = 23903.683571437483
Optimization restart 2/10, f = 23903.683571434958
Optimization restart 3/10, f = 23903.683571438636
Optimization restart 4/10, f = 23903.683571540798
Optimization restart 5/10, f = 23903.683571434412
Optimization restart 6/10, f = 23903.683571434354
Optimization restart 7/10, f = 23903.68357143426
Optimization restart 8/10, f = 23903.683571629612
Optimization restart 9/10, f = 23903.68357143458
Optimization restart 10/10, f = 23903.683572364767
  index  |  GP_regression.rbf.lengthscale  |  constraints  |  priors
  [0]    |                     0.95540868  |      +ve      |        
  [1]    |                     1.29307033  |      +ve      |        
  [2]    |                     1.64061195  |      +ve      |        
  [3]    |                     0.40970299  |      +ve      |        
  [4]    |                     0.86201309  |      +ve      |        
  [5]    |                     0.39536117  |      +ve      |        
  [6]    |        

(<GPy.models.gp_regression.GPRegression at 0x1612bba40>, -3.2254024838153352)

In [10]:
restarts = 20 
test_perc = 0.2

# Initialize dictionaries to store R2 scores and models
r2_scores = {}
fitted_models = {}

for key, output_data in dataframes.items():
    model, r2 = emulate_RBF(input=filtered_input, output=output_data, percentage=test_perc, restarts=restarts, suppress=True)
    r2_scores[key] = r2
    fitted_models[key] = model


# Convert the dictionaries to a DataFrame
results_df = pd.DataFrame({
    'R2_Score': pd.Series(r2_scores),
    'Model': pd.Series(fitted_models)
})

# Now `results_df` will be a DataFrame with column names as indices, R2 scores, and models
print(results_df)

# Save the DataFrame to a CSV file (models will not be saved in this step)
results_df.to_csv(f'{main_path}/Outputs/Output_{n_runs}/RBF_models_and_r2_scores_{n_runs}.csv')

# To save the DataFrame with models, use pickle
results_df.to_pickle(f'{main_path}/Outputs/Output_{n_runs}/RBF_models_and_r2_scores_{n_runs}.csv')

                 R2_Score                                              Model
press_only_PC1  -0.000769  \nName : GP regression\nObjective : 14719.5881...
press_CO_PC1    -0.000769  \nName : GP regression\nObjective : 14727.4742...
press_CO_dt_PC1 -0.000769  \nName : GP regression\nObjective : 14727.7349...
press_only_PC2  -0.000079  \nName : GP regression\nObjective : 8492.21654...
press_CO_PC2    -0.000077  \nName : GP regression\nObjective : 8509.33348...
press_CO_dt_PC2 -0.000073  \nName : GP regression\nObjective : 8519.94982...
press_only_PC3  -0.002563  \nName : GP regression\nObjective : 6248.82929...
press_CO_PC3    -0.002560  \nName : GP regression\nObjective : 6248.93914...
press_CO_dt_PC3 -0.002567  \nName : GP regression\nObjective : 6251.06582...
CO              -5.314388  \nName : GP regression\nObjective : 12736.4519...
mean_press      -3.256388  \nName : GP regression\nObjective : 23185.8937...
max_press       -3.225402  \nName : GP regression\nObjective : 23903.6835...

### Conduct Sensitivity Analysis

Define a `dict` defining the number of inputs, the names of the inputs, and the bounds on each input:

In [12]:
problem = {
    'num_vars': len(relevant_columns),
    'names': relevant_columns,
    'bounds' : filtered_input[relevant_columns].describe().loc[['min', 'max']].T.values
}

problem

{'num_vars': 37,
 'names': ['# sas.r',
  'sas.c',
  'sas.l',
  'sat.r',
  'sat.c',
  'sat.l',
  'svn.r',
  'svn.c',
  'pas.r',
  'pas.c',
  'pas.l',
  'pat.r',
  'pat.c',
  'pat.l',
  'pvn.r',
  'pvn.c',
  'ao.CQ',
  'mi.CQ',
  'po.CQ',
  'ti.CQ',
  'lv.E_pas',
  'lv.E_act',
  'lv.v_ref',
  'lv.k_pas',
  'la.E_pas',
  'la.E_act',
  'la.v_ref',
  'la.k_pas',
  'rv.E_pas',
  'rv.E_act',
  'rv.v_ref',
  'rv.k_pas',
  'ra.E_pas',
  'ra.E_act',
  'ra.v_ref',
  'ra.k_pas',
  'T'],
 'bounds': array([[1.50018981e-03, 4.49980116e-03],
        [4.00184438e-02, 1.19997578e-01],
        [3.10057612e-05, 9.29879954e-05],
        [5.35192001e-01, 1.60499507e+00],
        [8.00060226e-01, 2.39964639e+00],
        [8.50266717e-04, 2.54984306e-03],
        [3.75003130e-02, 1.12487141e-01],
        [1.02545471e+01, 3.07457828e+01],
        [1.00044833e-03, 2.99973891e-03],
        [9.00285912e-02, 2.69982353e-01],
        [2.60059113e-05, 7.79960381e-05],
        [1.55065605e-01, 1.23992814e+00],
      

As we are performing a Sobol sensitivity analysis, we should generate samples using the Saltelli sampler.

In [13]:
param_values = saltelli.sample(problem, 1024, calc_second_order=True)
param_values.shape

(77824, 37)

In [23]:
# Select Emulator
pca1_emulator = results_df['Model']['press_CO_dt_PC1']
pca1_emulator

GP_regression.,value,constraints,priors
rbf.variance,40.38747760161712,+ve,
rbf.lengthscale,"(37,)",+ve,
Gaussian_noise.variance,54.06987988465363,+ve,


In [29]:
predicted_values = pca1_emulator.predict(param_values)
predicted_values[0]

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [53]:
predicted_values[0].shape

(77824, 1)

`sobol.analyze` - will compute first, second, and total-order indices.

In [50]:
sobol_indices_pca = sobol.analyze(problem, test, calc_second_order=True),

sobol_indices_pca

({'S1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]),
  'S1_conf': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]),
  'ST': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]),
  'ST_conf': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]),
  'S2': array([[nan,  0.,  0., ...,  0.,  0.,  0.],
         [nan, nan,  0., ...,  0.,  0.,  0.],
         [nan, nan, nan, ...,  0.,  0.,  0.],
         ...,
         [nan, nan, nan, ..., nan,  0.,  0.],
         [nan, nan, nan, ..., nan, nan,  0.],
        

Here, *S1*, *ST* and *ST* cumulative sum is being calculated and the results are saved to file.

In [ ]:
for i in range(4):
    os.system(f'mkdir -p {main_path}/results/pca{i+1}')

    S1 = pd.DataFrame(sobol_indices_pca[i]['S1'], index=relevant_columns, columns=['S1'])
    S1.sort_values('S1', inplace=True, ascending=False)
    S1.to_csv(f'{main_path}/results/pca{i+1}/s1_{n_runs}.csv')
    
    ST = pd.DataFrame(sobol_indices_pca[i]['ST'], index=relevant_columns, columns=['ST'])
    ST.sort_values('ST', inplace=True, ascending=False)
    ST.to_csv(f'{main_path}/results/pca{i+1}/st_{n_runs}.csv')
    
    ST_cumsum = ST.cumsum() / ST.cumsum().iloc[-1]
    ST_cumsum.to_csv(f'{main_path}/results/pca{i+1}/st_cumsum_{n_runs}.csv')

Pie chart of ST results:

In [ ]:
sobol_indices_pca = sobol.analyze(problem, Y_pca1, calc_second_order=True)

ST = pd.DataFrame(sobol_indices_pca['ST'], index=relevant_columns, columns=['ST'])
ST.sort_values('ST', inplace=True, ascending=False)

labels = ST.index
sizes = ST['ST']

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=180)
plt.title('Sensitivity Analysis Results')
plt.show()